***`Soha Pervez (50508730) and Huy Doan (151221835)`***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***Importing Libraries***

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import math 
from pandas.core import shared_docs
import scipy.stats

# visualization
import seaborn as sns

# similarity
from sklearn.metrics.pairwise import cosine_similarity

Read in Rating and Movie Data set 

In [ ]:
rating = pd.read_csv("/content/drive/MyDrive/DATA.ML.360/ratings.csv")
movies = pd.read_csv("/content/drive/MyDrive/DATA.ML.360/movies.csv")

movies.head(10)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
rating.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


# Exploring the Datasets

In [ ]:
print("There are", rating.shape[0], "rows (total ratings) in the dataset")
print("There are", rating.userId.nunique(), "unique users in the dataset")
print("There are", rating.movieId.nunique(), "unique movies in the dataset")
print("There are", rating.rating.nunique(), "unique ratings in the dataset")
print("The ratings are ",sorted(rating.rating.unique()))

There are 100836 rows (total ratings) in the dataset
There are 610 unique users in the dataset
There are 9724 unique movies in the dataset
There are 10 unique ratings in the dataset
The ratings are  [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


Merge Movies + Rating

In [ ]:
df = pd.merge(rating, movies, on='movieId', how='inner')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Create a User Item Matrix

In [ ]:
matrix_user_df = df.pivot_table(index='userId', columns='title', values='rating')
matrix_user_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Normalize user-item matrix
matrix_user_norm_df = matrix_user_df.subtract(matrix_user_df.mean(axis=1), axis = 'rows')
matrix_user_norm_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.366379,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Identify Similar Users using Pearson correlation

In [ ]:
user_similarity_df = matrix_user_norm_df.T.corr()
user_similarity_df.head()


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,0.079819,0.207983,0.268749,-0.291636,-0.118773,0.469668,0.918559,-0.037987,...,0.091574,-1.183502e-17,-0.061503,-0.407556,-0.164871,0.066378,0.174557,0.268070,-0.175412,-0.032086
2,NaN,1.0,NaN,NaN,NaN,NaN,-0.991241,NaN,NaN,0.037796,...,-0.387347,NaN,-1.000000,NaN,NaN,0.583333,NaN,-0.125000,NaN,0.623288
3,0.079819,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.433200,NaN,NaN,-0.791334,-0.333333,-0.395092,NaN,0.569562
4,0.207983,NaN,NaN,1.000000,-0.336525,0.148498,0.542861,0.117851,NaN,0.485794,...,-0.222113,3.966413e-01,0.090090,-0.080296,0.400124,0.144603,0.116518,-0.170501,-0.277350,-0.043786
5,0.268749,NaN,NaN,-0.336525,1.000000,0.043166,0.158114,0.028347,NaN,-0.777714,...,0.000000,1.533034e-01,0.234743,0.067791,-0.364156,0.244321,0.231080,-0.020546,0.384111,0.040582


In [ ]:
# Pick a user ID
picked_userid = 1

# Remove picked user ID from the users list
user_similarity_df.drop(index=picked_userid, inplace = True)
user_similarity_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
2,NaN,1.000000,NaN,NaN,NaN,NaN,-0.991241,NaN,NaN,0.037796,...,-0.387347,NaN,-1.000000,NaN,NaN,0.583333,NaN,-0.125000,NaN,0.623288
3,0.079819,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.433200,NaN,NaN,-0.791334,-0.333333,-0.395092,NaN,0.569562
4,0.207983,NaN,NaN,1.000000,-0.336525,0.148498,0.542861,0.117851,NaN,0.485794,...,-0.222113,0.396641,0.090090,-0.080296,0.400124,0.144603,0.116518,-0.170501,-0.277350,-0.043786
5,0.268749,NaN,NaN,-0.336525,1.000000,0.043166,0.158114,0.028347,NaN,-0.777714,...,0.000000,0.153303,0.234743,0.067791,-0.364156,0.244321,0.231080,-0.020546,0.384111,0.040582
6,-0.291636,NaN,NaN,0.148498,0.043166,1.000000,-0.126595,-0.200062,NaN,0.957427,...,-0.292770,-0.027495,-0.112141,-0.090297,0.100735,-0.049192,0.255639,0.125428,0.193649,0.115580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.066378,0.583333,-0.791334,0.144603,0.244321,-0.049192,0.137771,0.253582,0.572700,-0.382955,...,0.290490,0.140613,0.318473,0.682949,0.167062,1.000000,0.114191,0.240842,0.533002,0.389185
607,0.174557,NaN,-0.333333,0.116518,0.231080,0.255639,0.402792,0.251280,NaN,-0.241121,...,0.698241,0.217210,0.192787,0.035806,-0.299641,0.114191,1.000000,0.200814,0.190117,0.106605
608,0.268070,-0.125000,-0.395092,-0.170501,-0.020546,0.125428,0.008081,0.434423,0.336625,-0.571043,...,0.473967,0.297646,0.086423,0.053986,-0.075673,0.240842,0.200814,1.000000,0.488929,0.147606


In [ ]:
# Number of similar users
n = 10

# User similarity threashold
user_similarity_threshold = 0.3

# Get top n similar users
top_10_similar_users_panda_series = user_similarity_df[user_similarity_df[picked_userid]>user_similarity_threshold][picked_userid].sort_values(ascending=False)[:n]

# Print out top n similar users
print(f'The similar users for user 1 are', top_10_similar_users_panda_series)

The similar users for user 1 are userId
598    1.000000
333    1.000000
146    1.000000
106    1.000000
550    1.000000
473    0.962250
511    0.925820
9      0.918559
13     0.878310
366    0.872872
Name: 1, dtype: float64


Recommend Items

In [ ]:
# Movies that similar users watched. Remove movies that none of the similar users have watched
similar_user_movies_df = matrix_user_norm_df[matrix_user_norm_df.index.isin(top_10_similar_users_panda_series.index)].dropna(axis=1, how='all')
similar_user_movies_df

title,101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Years a Slave (2013),13 Going on 30 (2004),1408 (2007),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",2001: A Space Odyssey (1968),21 (2008),21 Grams (2003),...,Vanilla Sky (2001),WALL·E (2008),Waking Life (2001),What Dreams May Come (1998),What Lies Beneath (2000),What's Eating Gilbert Grape (1993),Witness (1985),"Wolf of Wall Street, The (2013)",X-Men (2000),xXx (2002)
userId,,,,,,,,,,,,,,,,,,,,,
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.73913,NaN,NaN,-2.26087
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.354839,NaN,NaN,NaN,0.354839,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,-0.140625,0.859375,-0.640625,NaN,-0.140625,NaN,-1.640625,NaN,0.359375,-0.140625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.64,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,NaN,NaN
366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.581081,NaN,0.581081,-0.418919,NaN,NaN,NaN,NaN,NaN,NaN
511,NaN,NaN,NaN,-0.125,NaN,-0.625,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.107143,NaN,NaN,NaN,NaN,NaN,-0.107143,NaN,NaN


In [ ]:
# A dictionary to store item scores:
item_score = {}

#looping through each movie watched by similar users
for movie_name in similar_user_movies_df.columns:
  #get all the ratings from all the similar users for that one movie
  all_users_ratings_for_that_movie = similar_user_movies_df[movie_name]
  #score total
  total = 0
  #number of scores from different users
  count = 0
  #looping through each similar users
  for user_id in top_10_similar_users_panda_series.index:
    #if the rating from that userid for that movie not null
    if pd.isna(all_users_ratings_for_that_movie[user_id]) == False:
      #score = user similarity score + userid movie rating
      score = top_10_similar_users_panda_series[user_id] * all_users_ratings_for_that_movie[user_id]
      #add that to score so far for that movie
      total += score
      #iterate the count
      count += 1
  #final score = average of all the scores
  item_score[movie_name] = total/count

# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])

# Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

# Select top m -20 movies to recommend for user 1
m = 20
ranked_item_score.head(m)


,movie,movie_score
193,One Flew Over the Cuckoo's Nest (1975),2.360000
44,Bossa Nova (2000),2.360000
12,3-Iron (Bin-jip) (2004),1.859375
272,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),1.859375
31,Back to the Future (1985),1.597493
19,Adaptation (2002),1.597493
58,Citizen Kane (1941),1.597493
171,Love's Labour's Lost (2000),1.360000
169,Loser (2000),1.360000
15,50 First Dates (2004),1.359375


#Item-Based Approach

***create a user item matrix***

In [ ]:
# Create user-item matrix
matrix_item = df.pivot_table(index='title', columns='userId', values='rating')
matrix_item.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***matrix data Normalization***

In [ ]:
# Normalize user-item matrix
matrix_item_norm = matrix_item.subtract(matrix_item.mean(axis=1), axis = 0)
matrix_item_norm

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.136364,NaN,NaN,NaN,NaN,0.636364,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.770833,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.729167,NaN,-0.770833
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.500000


***calculate similarity among items using both pearson and cosine***

In [ ]:
item_similarity_pearson = matrix_item_norm.T.corr()
item_similarity_pearson.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_item_similarity_cosine = pd.DataFrame(cosine_similarity(matrix_item_norm.fillna(0.0000001)))
df_item_similarity_cosine

,0,1,2,3,4,5,6,7,8,9,...,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718
0,1.000000e+00,9.983580e-01,9.975353e-01,9.983580e-01,1.739262e-06,9.983580e-01,4.849432e-07,9.983580e-01,9.979238e-04,2.040971e-06,...,9.983580e-01,1.167014e-03,6.254965e-03,9.991786e-01,9.983580e-01,6.452161e-07,6.738618e-03,2.026347e-02,5.005109e-07,9.983580e-01
1,9.983580e-01,1.000000e+00,9.991786e-01,9.983580e-01,1.739262e-06,9.983580e-01,4.849432e-07,9.983580e-01,3.393780e-07,2.040971e-06,...,9.983580e-01,1.121121e-06,2.273477e-06,9.975353e-01,9.983580e-01,6.452161e-07,5.113676e-07,2.447532e-06,5.005109e-07,9.983580e-01
2,9.975353e-01,9.991786e-01,1.000000e+00,9.975353e-01,1.737824e-06,9.975353e-01,-2.651919e-03,9.975353e-01,3.390579e-07,2.039256e-06,...,9.975353e-01,1.120175e-06,2.271591e-06,9.967105e-01,9.975353e-01,6.446464e-07,5.109131e-07,2.445488e-06,5.000631e-07,9.975353e-01
3,9.983580e-01,9.983580e-01,9.975353e-01,1.000000e+00,-2.865167e-02,9.983580e-01,4.849432e-07,9.983580e-01,3.393780e-07,2.040971e-06,...,9.983580e-01,1.121121e-06,2.273477e-06,9.975353e-01,9.983580e-01,6.452161e-07,5.113676e-07,2.447532e-06,5.005109e-07,9.983580e-01
4,1.739262e-06,1.739262e-06,1.737824e-06,-2.865167e-02,1.000000e+00,1.739262e-06,8.447929e-13,1.739262e-06,5.911677e-13,3.555564e-12,...,1.739262e-06,1.953092e-12,3.960655e-12,1.737824e-06,1.739262e-06,1.123979e-12,8.908075e-13,4.263854e-12,8.718899e-13,1.739262e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,6.452161e-07,6.452161e-07,6.446464e-07,6.452161e-07,1.123979e-12,6.452161e-07,6.069914e-02,-1.498230e-03,-7.527707e-02,2.947369e-02,...,6.452161e-07,7.242529e-13,1.469123e-12,6.446464e-07,6.452161e-07,1.000000e+00,3.784586e-02,1.581409e-12,-3.410932e-02,6.452161e-07
9715,6.738618e-03,5.113676e-07,5.109131e-07,5.113676e-07,8.908075e-13,5.113676e-07,-3.527099e-03,5.113676e-07,1.558884e-01,2.511178e-01,...,1.547995e-02,-2.067568e-01,-4.854178e-03,4.739290e-03,5.113676e-07,3.784586e-02,1.000000e+00,2.449297e-01,1.854832e-02,5.113676e-07
9716,2.026347e-02,2.447532e-06,2.445488e-06,2.447532e-06,4.263854e-12,2.447532e-06,1.188653e-12,2.447532e-06,1.107823e-01,5.003227e-12,...,2.447532e-06,-2.014023e-01,7.715174e-02,2.028013e-02,2.447532e-06,1.581409e-12,2.449297e-01,1.000000e+00,1.226683e-12,2.447532e-06
9717,5.005109e-07,5.005109e-07,5.000631e-07,5.005109e-07,8.718899e-13,5.005109e-07,1.683583e-02,1.156187e-03,4.333034e-02,1.292903e-01,...,1.156187e-03,5.617753e-13,1.139610e-12,5.000631e-07,5.005109e-07,-3.410932e-02,1.854832e-02,1.226683e-12,1.000000e+00,5.005109e-07


In [ ]:
df_item_similarity_cosine.columns = item_similarity_pearson.columns
df_item_similarity_cosine.index = item_similarity_pearson.index

In [ ]:
df_item_similarity_cosine

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000e+00,9.983580e-01,9.975353e-01,9.983580e-01,1.739262e-06,9.983580e-01,4.849432e-07,9.983580e-01,9.979238e-04,2.040971e-06,...,9.983580e-01,1.167014e-03,6.254965e-03,9.991786e-01,9.983580e-01,6.452161e-07,6.738618e-03,2.026347e-02,5.005109e-07,9.983580e-01
'Hellboy': The Seeds of Creation (2004),9.983580e-01,1.000000e+00,9.991786e-01,9.983580e-01,1.739262e-06,9.983580e-01,4.849432e-07,9.983580e-01,3.393780e-07,2.040971e-06,...,9.983580e-01,1.121121e-06,2.273477e-06,9.975353e-01,9.983580e-01,6.452161e-07,5.113676e-07,2.447532e-06,5.005109e-07,9.983580e-01
'Round Midnight (1986),9.975353e-01,9.991786e-01,1.000000e+00,9.975353e-01,1.737824e-06,9.975353e-01,-2.651919e-03,9.975353e-01,3.390579e-07,2.039256e-06,...,9.975353e-01,1.120175e-06,2.271591e-06,9.967105e-01,9.975353e-01,6.446464e-07,5.109131e-07,2.445488e-06,5.000631e-07,9.975353e-01
'Salem's Lot (2004),9.983580e-01,9.983580e-01,9.975353e-01,1.000000e+00,-2.865167e-02,9.983580e-01,4.849432e-07,9.983580e-01,3.393780e-07,2.040971e-06,...,9.983580e-01,1.121121e-06,2.273477e-06,9.975353e-01,9.983580e-01,6.452161e-07,5.113676e-07,2.447532e-06,5.005109e-07,9.983580e-01
'Til There Was You (1997),1.739262e-06,1.739262e-06,1.737824e-06,-2.865167e-02,1.000000e+00,1.739262e-06,8.447929e-13,1.739262e-06,5.911677e-13,3.555564e-12,...,1.739262e-06,1.953092e-12,3.960655e-12,1.737824e-06,1.739262e-06,1.123979e-12,8.908075e-13,4.263854e-12,8.718899e-13,1.739262e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),6.452161e-07,6.452161e-07,6.446464e-07,6.452161e-07,1.123979e-12,6.452161e-07,6.069914e-02,-1.498230e-03,-7.527707e-02,2.947369e-02,...,6.452161e-07,7.242529e-13,1.469123e-12,6.446464e-07,6.452161e-07,1.000000e+00,3.784586e-02,1.581409e-12,-3.410932e-02,6.452161e-07
xXx (2002),6.738618e-03,5.113676e-07,5.109131e-07,5.113676e-07,8.908075e-13,5.113676e-07,-3.527099e-03,5.113676e-07,1.558884e-01,2.511178e-01,...,1.547995e-02,-2.067568e-01,-4.854178e-03,4.739290e-03,5.113676e-07,3.784586e-02,1.000000e+00,2.449297e-01,1.854832e-02,5.113676e-07
xXx: State of the Union (2005),2.026347e-02,2.447532e-06,2.445488e-06,2.447532e-06,4.263854e-12,2.447532e-06,1.188653e-12,2.447532e-06,1.107823e-01,5.003227e-12,...,2.447532e-06,-2.014023e-01,7.715174e-02,2.028013e-02,2.447532e-06,1.581409e-12,2.449297e-01,1.000000e+00,1.226683e-12,2.447532e-06


# Making basic prediction for movie 'Steve Jobs (2015)'

In [ ]:
#Set a user ID
picked_userid = 1

# Pick a movie
picked_movie = 'American Pie (1999)'

# Movies that the target user has watched
picked_userid_watched = pd.DataFrame(matrix_item_norm[picked_userid].dropna(axis=0, how='all')\
                          .sort_values(ascending=False))\
                          .reset_index()\
                          .rename(columns={1:'rating'})

picked_userid_watched

,title,rating
0,Scream 3 (2000),2.327586
1,Canadian Bacon (1995),2.136364
2,"Newton Boys, The (1998)",2.000000
3,Gulliver's Travels (1939),2.000000
4,Superman II (1980),1.979592
...,...,...
227,"Shining, The (1980)",-1.082569
228,"Mummy, The (1999)",-1.120690
229,Pulp Fiction (1994),-1.197068
230,Psycho (1960),-2.036145


In [ ]:
# Similarity score of the movie Steve Jobs with all the other movies
picked_movie_similarity_score = df_item_similarity_cosine[[picked_movie]].reset_index().rename(columns={'American Pie (1999)':'similarity_score', 'index':'title'})
picked_movie_similarity_score


title,title,similarity_score
0,'71 (2014),1.944129e-07
1,'Hellboy': The Seeds of Creation (2004),1.944129e-07
2,'Round Midnight (1986),1.941882e-07
3,'Salem's Lot (2004),1.944129e-07
4,'Til There Was You (1997),3.385786e-13
...,...,...
9714,eXistenZ (1999),-2.561714e-02
9715,xXx (2002),1.312479e-02
9716,xXx: State of the Union (2005),2.370408e-02
9717,¡Three Amigos! (1986),5.632404e-02


In [ ]:
# Rank the similarities between the movies user 1 rated and Steve Jobs (2015)
n = 5
picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                            right=picked_movie_similarity_score, 
                                            on='title', 
                                            how='inner')\
                                     .sort_values('similarity_score', ascending=False)[:5]

# movies that user 1 watch with highest similarity + rating
picked_userid_watched_similarity

,title,rating,similarity_score
118,Road Trip (2000),0.914634,0.294596
144,Rush Hour (1998),0.738095,0.242107
177,Lethal Weapon (1987),0.326667,0.235252
121,Star Wars: Episode I - The Phantom Menace (1999),0.892857,0.223208
173,Grosse Pointe Blank (1997),0.389831,0.210713


In [ ]:
# Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                    weights=picked_userid_watched_similarity['similarity_score']), 6)

print(f'The predicted rating for {picked_movie} by user {picked_userid} is {predicted_rating}' )

The predicted rating for American Pie (1999) by user 1 is 0.66875


# Recommend 20 Movies for User 1

In [ ]:
# Item-based recommendation function
def item_based_rec(picked_userid, number_of_similar_items, number_of_recommendations):
  import operator
  # Movies that the target user has not watched
  picked_userid_unwatched = pd.DataFrame(matrix_item_norm[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['title'].values.tolist()

  # Movies that the target user has watched
  picked_userid_watched = pd.DataFrame(matrix_item_norm[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={picked_userid:'rating'})
  
  # Dictionary to save the unwatched movie and predicted rating pair
  rating_prediction ={}  

  # Loop through unwatched movies          
  for picked_movie in picked_userid_unwatched: 
    # Calculate the similarity score of the picked movie iwth other movies
    # picked_movie_similarity_score = item_similarity_pearson[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    picked_movie_similarity_score = df_item_similarity_cosine[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})

    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                                right=picked_movie_similarity_score, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]

    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
    # predicted_rating = round(np.average(picked_userid_watched_similarity['rating']),6)
    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    
    # Save the predicted rating in the dictionary
    rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies
  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

# Get recommendations
recommended_movie = item_based_rec(picked_userid=1, number_of_similar_items=5, number_of_recommendations =20)
recommended_movie

[('Valerian and the City of a Thousand Planets (2017)', 1.631481),
 ('Firewalker (1986)', 1.617744),
 ('Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)', 1.617715),
 ('Crow: Salvation, The (2000)', 1.615339),
 ('Baby... Secret of the Lost Legend (1985)', 1.599267),
 ('Ant Bully, The (2006)', 1.595968),
 ('Curse of the Jade Scorpion, The (2001)', 1.59194),
 ('Deuce Bigalow: European Gigolo (2005)', 1.579533),
 ('Jackass: The Movie (2002)', 1.577884),
 ('Soul Plane (2004)', 1.575613),
 ('Swan Princess, The (1994)', 1.565838),
 ('Hustle & Flow (2005)', 1.564681),
 ('Doctor Who: Last Christmas (2014)', 1.561683),
 ('Doctor Who: The Doctor, the Widow and the Wardrobe (2011)', 1.561683),
 ('Doctor Who: The Husbands of River Song (2015)', 1.561683),
 ('Doctor Who: The Runaway Bride (2007)', 1.561683),
 ('Doctor Who: The Time of the Doctor (2013)', 1.561683),
 ('Doctor Who: Voyage Of The Damned (2007)', 1.561683),
 ('Kangaroo Jack (2003)', 1.561683),
 ('Safe (2012)', 1.561683)]

***Input number of users to recommend***

In [ ]:
number_of_users = int(input('How many users to recommend?:'))

How many users to recommend?:3


***Movies recommend for each user and put in a list***

In [ ]:
output_list = []
for user in range(1,number_of_users+1):
    recommended_movie = item_based_rec(picked_userid=user, number_of_similar_items=5, number_of_recommendations =5)
    output_list.append(recommended_movie)

In [ ]:
output_list

[[('Valerian and the City of a Thousand Planets (2017)', 1.631481),
  ('Firewalker (1986)', 1.617744),
  ('Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)', 1.617715),
  ('Crow: Salvation, The (2000)', 1.615339),
  ('Baby... Secret of the Lost Legend (1985)', 1.599267)],
 [('Alvin and the Chipmunks: The Squeakquel (2009)', 1.446423),
  ('Surviving Christmas (2004)', 1.445179),
  ('Infamous (2006)', 1.433685),
  ('Nancy Drew (2007)', 1.309797),
  ('Way of the Dragon, The (a.k.a. Return of the Dragon) (Meng long guo jiang) (1972)',
   1.275156)],
 [('Jurassic World: Fallen Kingdom (2018)', 1.815947),
  ('Friday the 13th Part V: A New Beginning (1985)', 1.815917),
  ('Keeping Up with the Joneses (2016)', 1.815828),
  ('Cannibal Holocaust (1980)', 1.516992),
  ('Buffy the Vampire Slayer (1992)', 1.460794)]]

***Convert the list of all recommended movies and scores to dataframe:***

In [ ]:
gr_rec_movies_dataframe =  pd.DataFrame([t for lst in output_list for t in lst])
gr_rec_movies_dataframe = gr_rec_movies_dataframe.rename(columns={0: 'movie', 1: 'score'})
gr_rec_movies_dataframe

,movie,score
0,Valerian and the City of a Thousand Planets (2...,1.631481
1,Firewalker (1986),1.617744
2,"Fist of Fury (Chinese Connection, The) (Jing w...",1.617715
3,"Crow: Salvation, The (2000)",1.615339
4,Baby... Secret of the Lost Legend (1985),1.599267
5,Alvin and the Chipmunks: The Squeakquel (2009),1.446423
6,Surviving Christmas (2004),1.445179
7,Infamous (2006),1.433685
8,Nancy Drew (2007),1.309797
9,"Way of the Dragon, The (a.k.a. Return of the D...",1.275156


***Average Approach***

In [ ]:
gr_rec_movies_dataframe_mean = gr_rec_movies_dataframe.groupby(['movie']).mean()
final_recommendation_mean = gr_rec_movies_dataframe_mean.sort_values(by=['score'], ascending=False)
final_recommendation_mean.head(5)

,score
movie,
Jurassic World: Fallen Kingdom (2018),1.815947
Friday the 13th Part V: A New Beginning (1985),1.815917
Keeping Up with the Joneses (2016),1.815828
Valerian and the City of a Thousand Planets (2017),1.631481
Firewalker (1986),1.617744


***Least Misery Min Approach***

In [ ]:
gr_rec_movies_dataframe_min_least_misery = gr_rec_movies_dataframe.groupby(['movie']).min()
final_recommendation_min = gr_rec_movies_dataframe_min_least_misery.sort_values(by=['score'], ascending=False)
final_recommendation_min.head(5)

,score
movie,
Jurassic World: Fallen Kingdom (2018),1.815947
Friday the 13th Part V: A New Beginning (1985),1.815917
Keeping Up with the Joneses (2016),1.815828
Valerian and the City of a Thousand Planets (2017),1.631481
Firewalker (1986),1.617744


# ***Recommendation for a group of three users***

In [ ]:
output_list_group_3 = []
user_id_lists = [1,6,7]
for user in user_id_lists:
    recommended_movie = item_based_rec(picked_userid=user, number_of_similar_items=5, number_of_recommendations =20)
    output_list_group_3.append(recommended_movie)
output_list_group_3

[[('Valerian and the City of a Thousand Planets (2017)', 1.631481),
  ('Firewalker (1986)', 1.617744),
  ('Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)', 1.617715),
  ('Crow: Salvation, The (2000)', 1.615339),
  ('Baby... Secret of the Lost Legend (1985)', 1.599267),
  ('Ant Bully, The (2006)', 1.595968),
  ('Curse of the Jade Scorpion, The (2001)', 1.59194),
  ('Deuce Bigalow: European Gigolo (2005)', 1.579533),
  ('Jackass: The Movie (2002)', 1.577884),
  ('Soul Plane (2004)', 1.575613),
  ('Swan Princess, The (1994)', 1.565838),
  ('Hustle & Flow (2005)', 1.564681),
  ('Doctor Who: Last Christmas (2014)', 1.561683),
  ('Doctor Who: The Doctor, the Widow and the Wardrobe (2011)', 1.561683),
  ('Doctor Who: The Husbands of River Song (2015)', 1.561683),
  ('Doctor Who: The Runaway Bride (2007)', 1.561683),
  ('Doctor Who: The Time of the Doctor (2013)', 1.561683),
  ('Doctor Who: Voyage Of The Damned (2007)', 1.561683),
  ('Kangaroo Jack (2003)', 1.561683),
  ('Safe (20

In [ ]:
len(output_list_group_3[1])

20

***Convert the list of all recommended movies and scores to dataframe:***

In [ ]:
group_movies_df =  pd.DataFrame([t for lst in output_list_group_3 for t in lst])
group_movies_df = group_movies_df.rename(columns={0: 'movie', 1: 'score'})
group_movies_df

,movie,score
0,Valerian and the City of a Thousand Planets (2...,1.631481
1,Firewalker (1986),1.617744
2,"Fist of Fury (Chinese Connection, The) (Jing w...",1.617715
3,"Crow: Salvation, The (2000)",1.615339
4,Baby... Secret of the Lost Legend (1985),1.599267
5,"Ant Bully, The (2006)",1.595968
6,"Curse of the Jade Scorpion, The (2001)",1.591940
7,Deuce Bigalow: European Gigolo (2005),1.579533
8,Jackass: The Movie (2002),1.577884
9,Soul Plane (2004),1.575613


***Average Approach***

In [ ]:
df_mean = group_movies_df.groupby(['movie']).mean()
recommendations_for_3 = df_mean.sort_values(by=['score'], ascending=False)
recommendations_for_3.head(20)

,score
movie,
Valerian and the City of a Thousand Planets (2017),1.631481
Barney's Great Adventure (1998),1.621275
Firewalker (1986),1.617744
"Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)",1.617715
"Crow: Salvation, The (2000)",1.615339
Mission: Impossible - Fallout (2018),1.609352
Baby... Secret of the Lost Legend (1985),1.599267
"Ant Bully, The (2006)",1.595968
"Curse of the Jade Scorpion, The (2001)",1.591940


***Least Misery Min Approach***

In [ ]:
df_min = group_movies_df.groupby(['movie']).min()
recommendation_for_3_min = df_min.sort_values(by=['score'], ascending=False)
recommendation_for_3_min.head(20)

,score
movie,
Valerian and the City of a Thousand Planets (2017),1.631481
Barney's Great Adventure (1998),1.621275
Firewalker (1986),1.617744
"Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)",1.617715
"Crow: Salvation, The (2000)",1.615339
Mission: Impossible - Fallout (2018),1.609352
Baby... Secret of the Lost Legend (1985),1.599267
"Ant Bully, The (2006)",1.595968
"Curse of the Jade Scorpion, The (2001)",1.591940


In [ ]:
from scipy.stats import pearsonr, kendalltau

# ***Kendall Distance***

In [ ]:
recommendations_for_3_list = recommendations_for_3['score'].values.tolist()
recommendations_for_3_list = recommendations_for_3_list[:20]
recommendations_for_3_list

[1.631481,
 1.621275,
 1.617744,
 1.617715,
 1.615339,
 1.609352,
 1.599267,
 1.595968,
 1.59194,
 1.579533,
 1.577884,
 1.565838,
 1.564681,
 1.561683,
 1.561683,
 1.561683,
 1.561683,
 1.561683,
 1.561683,
 1.561683]

In [ ]:
#recommendations for user 1

print("Recommended movies for user 1")
recommended_movies_user_1 = item_based_rec(picked_userid=11, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_1_df = pd.DataFrame(recommended_movies_user_1)
recommended_movies_user_1_df_list = recommended_movies_user_1_df[1].values.tolist()
recommended_movies_user_1_df_list

Recommended movies for user 1


[1.698066,
 1.668981,
 1.668981,
 1.580247,
 1.42841,
 1.399619,
 1.388347,
 1.358631,
 1.342548,
 1.339158,
 1.322701,
 1.32067,
 1.319942,
 1.319308,
 1.319308,
 1.319308,
 1.319308,
 1.319308,
 1.319308,
 1.319308]

In [ ]:
#let's create Kendall-Tau distance calculation to evaluate the recommendation of our grouping (against Average Aggregation method)

#User 11 and the group
tauAAVG_user1, p_valueA_user1 = kendalltau(recommended_movies_user_1_df_list, recommendations_for_3_list)

In [ ]:
print("Recommended movies for user 473")
recommended_movies_user_473 = item_based_rec(picked_userid=473, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_473_df = pd.DataFrame(recommended_movies_user_473)
recommended_movies_user_473_df_list = recommended_movies_user_473_df[1].values.tolist()
recommended_movies_user_473_df_list

Recommended movies for user 473


[1.132504,
 1.131973,
 1.129707,
 1.013514,
 1.013514,
 1.009629,
 0.994811,
 0.994811,
 0.994811,
 0.971577,
 0.963114,
 0.963114,
 0.963114,
 0.963114,
 0.963114,
 0.94336,
 0.916365,
 0.912421,
 0.886496,
 0.872073]

In [ ]:
tauAAVG_user473, p_valueA_user473 = kendalltau(recommended_movies_user_473_df_list, recommendations_for_3_list)

In [ ]:
print("Recommended movies for user 511")
recommended_movies_user_511 = item_based_rec(picked_userid=511, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_511_df = pd.DataFrame(recommended_movies_user_511)
recommended_movies_user_511_df_list = recommended_movies_user_511_df[1].values.tolist()
recommended_movies_user_511_df_list

Recommended movies for user 511


[1.005857,
 0.996499,
 0.992116,
 0.992116,
 0.992116,
 0.992116,
 0.991862,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874,
 0.975874]

In [ ]:
tauAAVG_user511, p_valueA_user511 = kendalltau(recommended_movies_user_511_df_list, recommendations_for_3_list)

In [ ]:
kendall_df = pd.DataFrame({'User': ['User 1', 'User 473', 'User 511'],
                    'TauAAVG': [tauAAVG_user1, tauAAVG_user473, tauAAVG_user511],
                    'PValue': [p_valueA_user1, p_valueA_user473, p_valueA_user511]})

kendall_df

#As the correlation coefficient value goes towards 0, the relationship between the two variables will be weaker. The direction of the relationship is indicated by the sign of the coefficient; a + sign indicates a positive relationship and a — sign indicates a negative relationship.

,User,TauAAVG,PValue
0,User 1,0.997037,1.082984e-08
1,User 473,0.904534,1.600471e-07
2,User 511,0.791972,2.935221e-05


In [ ]:
#propose a method that takes disagreements into account when computing suggestions for the group
#When there are overlapped movies, we will assume to be value = 0 if the other 2 users don't have that score

# ***Implementing Methods***

In [ ]:
group_movies_df.duplicated()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    False
dtype: bool

In [ ]:
# group_movies_df.duplicated()
new_group_df = pd.DataFrame(np.repeat(group_movies_df.values, 3, axis=0))
new_group_df.columns = group_movies_df.columns

In [ ]:
new_group_df

,movie,score
0,Valerian and the City of a Thousand Planets (2...,1.631481
1,Valerian and the City of a Thousand Planets (2...,1.631481
2,Valerian and the City of a Thousand Planets (2...,1.631481
3,Firewalker (1986),1.617744
4,Firewalker (1986),1.617744
...,...,...
175,Slumber Party Massacre II (1987),1.0972
176,Slumber Party Massacre II (1987),1.0972
177,Slumber Party Massacre III (1990),1.0972
178,Slumber Party Massacre III (1990),1.0972


In [ ]:
for i in range(0,180,3):
  i=i+1
  new_group_df.iloc[[i],[1]] = 0
  i=i+1
  new_group_df.iloc[[i],[1]] = 0

new_group_df

,movie,score
0,Valerian and the City of a Thousand Planets (2...,1.631481
1,Valerian and the City of a Thousand Planets (2...,0
2,Valerian and the City of a Thousand Planets (2...,0
3,Firewalker (1986),1.617744
4,Firewalker (1986),0
...,...,...
175,Slumber Party Massacre II (1987),0
176,Slumber Party Massacre II (1987),0
177,Slumber Party Massacre III (1990),1.0972
178,Slumber Party Massacre III (1990),0


In [ ]:
new_df_mean = new_group_df.groupby(['movie']).mean()
new_recommendations_for_3 = new_df_mean.sort_values(by=['score'], ascending=False)
new_recommendations_for_3.head(20)

,score
movie,
Valerian and the City of a Thousand Planets (2017),0.543827
Barney's Great Adventure (1998),0.540425
Firewalker (1986),0.539248
"Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)",0.539238
"Crow: Salvation, The (2000)",0.538446
Mission: Impossible - Fallout (2018),0.536451
Baby... Secret of the Lost Legend (1985),0.533089
"Ant Bully, The (2006)",0.531989
"Curse of the Jade Scorpion, The (2001)",0.530647


*Assignment 3*

In [ ]:
#For first iteration, we are doing the same steps as we did in assignment 2 to find the group recommendations
#for 3 users

In [ ]:
#Taking user 1 on random and as per assignment 1, similar users to user 1 are 473 and 511
output_list_group_3 = []
user_id_lists = [1,463,511]
for user in user_id_lists:
    recommended_movie = item_based_rec(picked_userid=user, number_of_similar_items=5, number_of_recommendations =20)
    output_list_group_3.append(recommended_movie)
output_list_group_3

[[('Valerian and the City of a Thousand Planets (2017)', 1.631481),
  ('Firewalker (1986)', 1.617744),
  ('Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)', 1.617715),
  ('Crow: Salvation, The (2000)', 1.615339),
  ('Baby... Secret of the Lost Legend (1985)', 1.599267),
  ('Ant Bully, The (2006)', 1.595968),
  ('Curse of the Jade Scorpion, The (2001)', 1.59194),
  ('Deuce Bigalow: European Gigolo (2005)', 1.579533),
  ('Jackass: The Movie (2002)', 1.577884),
  ('Soul Plane (2004)', 1.575613),
  ('Swan Princess, The (1994)', 1.565838),
  ('Hustle & Flow (2005)', 1.564681),
  ('Doctor Who: Last Christmas (2014)', 1.561683),
  ('Doctor Who: The Doctor, the Widow and the Wardrobe (2011)', 1.561683),
  ('Doctor Who: The Husbands of River Song (2015)', 1.561683),
  ('Doctor Who: The Runaway Bride (2007)', 1.561683),
  ('Doctor Who: The Time of the Doctor (2013)', 1.561683),
  ('Doctor Who: Voyage Of The Damned (2007)', 1.561683),
  ('Kangaroo Jack (2003)', 1.561683),
  ('Safe (20

In [ ]:
#converting into dataframe
group_movies_df =  pd.DataFrame([t for lst in output_list_group_3 for t in lst])
group_movies_df = group_movies_df.rename(columns={0: 'movie', 1: 'score'})
group_movies_df

,movie,score
0,Valerian and the City of a Thousand Planets (2...,1.631481
1,Firewalker (1986),1.617744
2,"Fist of Fury (Chinese Connection, The) (Jing w...",1.617715
3,"Crow: Salvation, The (2000)",1.615339
4,Baby... Secret of the Lost Legend (1985),1.599267
5,"Ant Bully, The (2006)",1.595968
6,"Curse of the Jade Scorpion, The (2001)",1.591940
7,Deuce Bigalow: European Gigolo (2005),1.579533
8,Jackass: The Movie (2002),1.577884
9,Soul Plane (2004),1.575613


In [ ]:
#using the average aggregation
df_mean = group_movies_df.groupby(['movie']).mean()
recommendations_for_3 = df_mean.sort_values(by=['score'], ascending=False)
recommendations_for_3.head(20)

,score
movie,
Valerian and the City of a Thousand Planets (2017),1.631481
Firewalker (1986),1.617744
"Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)",1.617715
"Crow: Salvation, The (2000)",1.615339
Baby... Secret of the Lost Legend (1985),1.599267
"Ant Bully, The (2006)",1.595968
"Curse of the Jade Scorpion, The (2001)",1.591940
Deuce Bigalow: European Gigolo (2005),1.579533
Jackass: The Movie (2002),1.577884


In [ ]:
def alpha_score(userARec, userBRec, userCRec, GroupRec):
  #Calculating the satisfaction score for every user
  satUser1 = np.sum(recommendations_for_3_list)/np.sum(recommended_movies_user_1_df_list)
  satUser473 = np.sum(recommendations_for_3_list)/np.sum(recommended_movies_user_473_df_list)
  satUser511 = np.sum(recommendations_for_3_list)/np.sum(recommended_movies_user_511_df_list)
  satUsersList = [satUser1, satUser473, satUser511]

  #Calculating the Group Satisfaction Score
  group_sat = np.mean(satUsersList)

  #Kendall Tau between the users so as to find out the disagreement among the users  
  ktAB, p_valAB = kendalltau(recommended_movies_user_1_df_list, recommended_movies_user_473_df_list)
  ktBC, p_valBC = kendalltau(recommended_movies_user_473_df_list, recommended_movies_user_511_df_list)
  ktAC, p_valAC = kendalltau(recommended_movies_user_1_df_list, recommended_movies_user_511_df_list)

  #Kendall Tau between users and group as done in assignment 2
  tauAAVG_user1, p_valueA_user1 = kendalltau(recommended_movies_user_1_df_list, recommendations_for_3_list)
  tauAAVG_user473, p_valueA_user473 = kendalltau(recommended_movies_user_473_df_list, recommendations_for_3_list)
  tauAAVG_user511, p_valueA_user511 = kendalltau(recommended_movies_user_511_df_list, recommendations_for_3_list)

  #Calculating the means for two scenarios ;- among users, between user individual list and generated movie list
  ktOnlyUsers = np.mean([ktAB, ktBC, ktAC])
  ktOverall = np.mean([tauAAVG_user1, tauAAVG_user473, tauAAVG_user511])

  #Group disagreement
  #Difference of the result of two scenarios
  alpha = abs(ktOnlyUsers - ktOverall)

  return satUser1, satUser473, satUser511, alpha, group_sat

In [ ]:
#movie recommendations for user 1
recommended_movie_1 = item_based_rec(picked_userid=1, number_of_similar_items=5, number_of_recommendations =20)
recommended_movie_1

[('Valerian and the City of a Thousand Planets (2017)', 1.631481),
 ('Firewalker (1986)', 1.617744),
 ('Fist of Fury (Chinese Connection, The) (Jing wu men) (1972)', 1.617715),
 ('Crow: Salvation, The (2000)', 1.615339),
 ('Baby... Secret of the Lost Legend (1985)', 1.599267),
 ('Ant Bully, The (2006)', 1.595968),
 ('Curse of the Jade Scorpion, The (2001)', 1.59194),
 ('Deuce Bigalow: European Gigolo (2005)', 1.579533),
 ('Jackass: The Movie (2002)', 1.577884),
 ('Soul Plane (2004)', 1.575613),
 ('Swan Princess, The (1994)', 1.565838),
 ('Hustle & Flow (2005)', 1.564681),
 ('Doctor Who: Last Christmas (2014)', 1.561683),
 ('Doctor Who: The Doctor, the Widow and the Wardrobe (2011)', 1.561683),
 ('Doctor Who: The Husbands of River Song (2015)', 1.561683),
 ('Doctor Who: The Runaway Bride (2007)', 1.561683),
 ('Doctor Who: The Time of the Doctor (2013)', 1.561683),
 ('Doctor Who: Voyage Of The Damned (2007)', 1.561683),
 ('Kangaroo Jack (2003)', 1.561683),
 ('Safe (2012)', 1.561683)]

In [ ]:
#movie recommendations for user 473
recommended_movie_473 = item_based_rec(picked_userid=473, number_of_similar_items=5, number_of_recommendations =20)
recommended_movie_473

[('Crew, The (2000)', 1.132504),
 ('Get on Up (2014)', 1.131973),
 ('I, the Jury (1982)', 1.129707),
 ('Fanny and Alexander (Fanny och Alexander) (1982)', 1.013514),
 ('Point of No Return (1993)', 1.013514),
 ('Taking Care of Business (1990)', 1.009629),
 ('The Town that Dreaded Sundown (2014)', 0.994811),
 ('Unknown Known, The (2013)', 0.994811),
 ('We Could Be King (2014)', 0.994811),
 ('Cocaine Cowboys (2006)', 0.971577),
 ('Avalon (1990)', 0.963114),
 ('Dressed to Kill (1980)', 0.963114),
 ('Otello (1986)', 0.963114),
 ('Solas (1999)', 0.963114),
 ('Son of the Bride (Hijo de la novia, El) (2001)', 0.963114),
 ("It's Such a Beautiful Day (2012)", 0.94336),
 ('End of Watch (2012)', 0.916365),
 ('Silence (2016)', 0.912421),
 ('Closet, The (Placard, Le) (2001)', 0.886496),
 ('Raise the Red Lantern (Da hong deng long gao gao gua) (1991)', 0.872073)]

In [ ]:
#movie recommendations for user 511
recommended_movie_511 = item_based_rec(picked_userid=511, number_of_similar_items=5, number_of_recommendations =20)
recommended_movie_511

[('Phantom of the Paradise (1974)', 1.005857),
 ('Reno 911!: Miami (2007)', 0.996499),
 ("Jason's Lyric (1994)", 0.992116),
 ("My Sister's Keeper (2009)", 0.992116),
 ('Rabbit Hole (2010)', 0.992116),
 ('Words, The (2012)', 0.992116),
 ('Hitcher, The (2007)', 0.991862),
 ('Bart Got a Room (2008)', 0.975874),
 ('Beauty Shop (2005)', 0.975874),
 ('Biker Boyz (2003)', 0.975874),
 ('Catch That Kid (2004)', 0.975874),
 ('Chaos Theory (2007)', 0.975874),
 ('Eulogy (2004)', 0.975874),
 ('Gods and Generals (2003)', 0.975874),
 ('Hannah Montana: The Movie (2009)', 0.975874),
 ('Hereafter (2010)', 0.975874),
 ('In the Name of the King: A Dungeon Siege Tale (2008)', 0.975874),
 ("King's Ransom (2005)", 0.975874),
 ('Like Mike (2002)', 0.975874),
 ('Man of the House (2005)', 0.975874)]

In [ ]:
#get satisfaction score of first iteration
satA1, satB1, satC1, alpha1, group_sat1= alpha_score(recommended_movie_1, recommended_movie_473, recommended_movie_511, recommendations_for_3_list)

In [ ]:
satA1

1.129966214436251

# Iteration 1

Divide the data into 2 chunks: 1 big chunk and 4 small chunks

In [ ]:
def alpha_score(userARec, userBRec, userCRec, GroupRec):
  #Calculating the satisfaction score for every user
  satUserA = np.sum(GroupRec)/np.sum(userARec)
  satUserB = np.sum(GroupRec)/np.sum(userBRec)
  satUserC = np.sum(GroupRec)/np.sum(userCRec)
  satUsersList = [satUserA, satUserB, satUserC]

  #Calculating the Group Satisfaction Score
  group_sat = np.mean(satUsersList)

  #Kendall Tau between the users so as to find out the disagreement among the users  
  ktAB, p_valAB = kendalltau(userARec, userBRec)
  ktBC, p_valBC = kendalltau(userBRec, userCRec)
  ktAC, p_valAC = kendalltau(userARec, userCRec)

  #Kendall Tau between users and group as done in assignment 2
  tauAAVG_userA, p_valueA_userA = kendalltau(userARec, GroupRec)
  tauAAVG_userB, p_valueA_userB = kendalltau(userBRec, GroupRec)
  tauAAVG_userC, p_valueA_userC = kendalltau(userCRec, GroupRec)

  #Calculating the means for two scenarios ;- among users, between user individual list and generated movie list
  ktOnlyUsers = np.mean([ktAB, ktBC, ktAC])
  ktOverall = np.mean([tauAAVG_userA, tauAAVG_userB, tauAAVG_userC])

  #Group disagreement
  #Difference of the result of two scenarios
  alpha = abs(ktOnlyUsers - ktOverall)

  return satUserA, satUserB, satUserC, alpha, group_sat

In [ ]:
import numpy as np
rating = pd.read_csv("/content/drive/MyDrive/DATA.ML.360/ratings.csv")
movies = pd.read_csv("/content/drive/MyDrive/DATA.ML.360/movies.csv")
df_assignment_3 = pd.merge(rating, movies, on='movieId', how='inner')
chunks = np.array_split(df_assignment_3, 2)
df_big_chunk = chunks[0]

In [ ]:
matrix_item_assignment_3 = df_big_chunk.pivot_table(index='title', columns='userId', values='rating')
matrix_item_norm_assignment_3 = matrix_item_assignment_3.subtract(matrix_item_assignment_3.mean(axis=1), axis = 0)
df_item_similarity_cosine_asign_3 = pd.DataFrame(cosine_similarity(matrix_item_norm_assignment_3.fillna(0.0000001)))
item_similarity_pearson_assignment_3 = matrix_item_norm_assignment_3.T.corr()
df_item_similarity_cosine_asign_3.columns = item_similarity_pearson_assignment_3.columns
df_item_similarity_cosine_asign_3.index = item_similarity_pearson_assignment_3.index

In [ ]:
# Item-based recommendation function
def item_based_rec_assignment_3(picked_userid, number_of_similar_items, number_of_recommendations):
  import operator
  # Movies that the target user has not watched
  picked_userid_unwatched = pd.DataFrame(matrix_item_norm_assignment_3[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['title'].values.tolist()

  # Movies that the target user has watched
  picked_userid_watched = pd.DataFrame(matrix_item_norm_assignment_3[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={picked_userid:'rating'})
  
  # Dictionary to save the unwatched movie and predicted rating pair
  rating_prediction ={}  

  # Loop through unwatched movies          
  for picked_movie in picked_userid_unwatched: 
    # Calculate the similarity score of the picked movie iwth other movies
    # picked_movie_similarity_score = item_similarity_pearson[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    picked_movie_similarity_score = df_item_similarity_cosine_asign_3[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})

    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                                right=picked_movie_similarity_score, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]

    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
    # predicted_rating = round(np.average(picked_userid_watched_similarity['rating']),6)
    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    
    # Save the predicted rating in the dictionary
    rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies
  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

In [ ]:
#recommendations for user 1

print("Recommended movies for user 1")
recommended_movies_user_1 = item_based_rec_assignment_3(picked_userid=1, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_1_df = pd.DataFrame(recommended_movies_user_1)
recommended_movies_user_1_df_list = recommended_movies_user_1_df[1].values.tolist()

Recommended movies for user 1


In [ ]:
print("Recommended movies for user 473")
recommended_movies_user_473 = item_based_rec_assignment_3(picked_userid=473, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_473_df = pd.DataFrame(recommended_movies_user_473)
recommended_movies_user_473_df_list = recommended_movies_user_473_df[1].values.tolist()

Recommended movies for user 473


In [ ]:
print("Recommended movies for user 511")
recommended_movies_user_511 = item_based_rec_assignment_3(picked_userid=511, number_of_similar_items=5, number_of_recommendations =20)
recommended_movies_user_511_df = pd.DataFrame(recommended_movies_user_511)
recommended_movies_user_511_df_list = recommended_movies_user_511_df[1].values.tolist()

Recommended movies for user 511


In [ ]:
output_list_group_3 = []
user_id_lists = [1,463,511]
for user in user_id_lists:
    recommended_movie = item_based_rec_assignment_3(picked_userid=user, number_of_similar_items=5, number_of_recommendations =20)
    output_list_group_3.append(recommended_movie)

In [ ]:
#converting into dataframe
group_movies_df =  pd.DataFrame([t for lst in output_list_group_3 for t in lst])
group_movies_df = group_movies_df.rename(columns={0: 'movie', 1: 'score'})

In [ ]:
#using the average aggregation
df_mean = group_movies_df.groupby(['movie']).mean()
recommendations_for_3 = df_mean.sort_values(by=['score'], ascending=False)

In [ ]:
recommendations_for_3_list = recommendations_for_3['score'].values.tolist()
recommendations_for_3_list_top20 = recommendations_for_3_list[:20]

In [ ]:
#get satisfaction score of first iteration
satA1, satB1, satC1, alpha1, group_sat1= alpha_score(recommended_movies_user_1_df_list, recommended_movies_user_473_df_list, recommended_movies_user_511_df_list, recommendations_for_3_list_top20)

In [ ]:
satA1, satB1, satC1, alpha1, group_sat1

(1.0, 2.163413623432453, 1.636631080731226, 0.0, 1.600014901387893)